In [1]:
%load_ext memory_profiler
import sys; sys.path.append("../src")

In [2]:
import lib.data as dataset
import os
import numpy as np
import pandas as pd
import pickle
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split

# About

This notebook looks at the exported pipeline that tpot generated in the model selection notebook. In this notebook that pipeline is modified to work with the existing dataset. That model is then exported in a serialized format to prevent having to retrain for invocations.

# Dataset

Source: [Heterogeneity Activity Recognition Data Set](https://archive.ics.uci.edu/ml/datasets/Heterogeneity+Activity+Recognition)

The dataset and accompanying research can befound at UCIs dataset repository. The copy used in this notebook was transformed from the original dataset in the `process_datasets.ipynb`

In [3]:
%time df = dataset.read_local_phones(); df.head(1)

CPU times: user 16.1 s, sys: 545 ms, total: 16.6 s
Wall time: 16.7 s


,arrival_time,target,user,x_accel,x_gyro,y_accel,y_gyro,z_accel,z_gyro
0,1424686733391,stand,g,-2.779668,0.015577,1.908179,-0.043371,8.927979,-0.014661


In [4]:
target_labels, target_uniques = pd.factorize(df.target)
user_labels, user_uniques = pd.factorize(df.user)

## Apply Changes and Select Subset
Factorizations are applied in this step, and a subset of the dataset is selected.

In testing Tpot took a very long time to complete a single generation of the full dataset. I opted to select 100,00 random samples instead. 

In [5]:
df = (
    df.assign(
        target=lambda _: target_labels,
        user=lambda _: user_labels
    )
    .drop(columns=["arrival_time"])
    .sample(n=125_000)
    .pipe(dataset.balance())
); df.head(1)

,target,user,x_accel,x_gyro,y_accel,y_gyro,z_accel,z_gyro
1385146,0,1,-2.607651,-0.06781,-1.014648,-0.047821,9.462204,0.365005


In [6]:
df.target.value_counts()

5    16660
4    16660
3    16660
2    16660
1    16660
0    16660
Name: target, dtype: int64

In [7]:
len(df)

99960

# Split Test and Train Set

In [8]:
X_train, X_test, y_train, y_test = (
    train_test_split(df.drop(columns=["target"]), df.target)
)

In [9]:
print(f"Train set size: {len(X_train)}; test set size: {len(X_test)}")

Train set size: 74970; test set size: 24990


# Pipeline

This pipeline was taken from `model_01.py`. None of the hyperparameters have been adjusted from the output.

In [10]:
len(df)

99960

In [11]:
pipeline = ExtraTreesClassifier(
    bootstrap=False, 
    criterion="gini", 
    max_features=0.3, 
    min_samples_leaf=1, 
    min_samples_split=4, 
    n_estimators=100
)

# Train

In [12]:
%time pipeline.fit(X_train, y_train)

CPU times: user 6.23 s, sys: 184 ms, total: 6.42 s
Wall time: 6.44 s


ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features=0.3,
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=4,
                     min_weight_fraction_leaf=0.0, n_estimators=100,
                     n_jobs=None, oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [13]:
%time results = pipeline.predict(X_test)

CPU times: user 765 ms, sys: 6 µs, total: 765 ms
Wall time: 766 ms


# Export Pipeline

In [14]:
pickle.dump(pipeline, open("../model/model_01.pkl", "wb"))

# Evaluate Exported Pipeline

In [15]:
%time rehydrated = pickle.load(open("../model/model_01.pkl", "rb"))

CPU times: user 146 ms, sys: 303 ms, total: 450 ms
Wall time: 450 ms


In [16]:
%time rehydrated.score(X_test, y_test)

CPU times: user 810 ms, sys: 72 µs, total: 810 ms
Wall time: 812 ms


0.833373349339736

In [ ]:
%memit rehydrated.score(X_test, y_test)

In [17]:
sample = X_test.sample(n=1)

peak memory: 2370.08 MiB, increment: 0.36 MiB


In [18]:
sample = X_test.sample(n=1)

In [19]:
%timeit rehydrated.predict(sample)

11.5 ms ± 198 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
%memit rehydrated.predict(sample)

peak memory: 2370.41 MiB, increment: 0.00 MiB
